In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
from torchtext.vocab import Vectors

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch

from itertools import chain

import pickle

import numpy as np
import math


import re
import os
from sklearn.model_selection import train_test_split
import pandas as pd

import itertools
import random
from IPython.display import display, HTML

import nltk
from nltk import stem
nltk.download('punkt')

def nltk_analyzer(text):
    stemmer = stem.LancasterStemmer()
    text = re.sub(re.compile(r'[!-\/:-@[-`{-~]'), ' ', text)
    text = stemmer.stem(text)
    text = text.replace('\n', '') # 改行削除
    text = text.replace('\t', '') # タブ削除
    morph = nltk.word_tokenize(text)
    return morph

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
imdb_dir = "drive/My Drive/Colab Notebooks/imdb_datasets/"
word_embedding_dir = "drive/My Drive/Colab Notebooks/word_embedding_models/"

with open(imdb_dir + 'train_df.pickle', 'rb') as f:
    train_df = pickle.load(f)
with open(imdb_dir + 'test_df.pickle', 'rb') as f:
    test_df = pickle.load(f)

col = ['text', 'label_id']
train_df[col].to_csv(imdb_dir + "train.tsv", sep="\t", index=None, header=False)
test_df[col].to_csv(imdb_dir + "test.tsv", sep="\t", index=None, header=False)

TEXT = data.Field(sequential=True, tokenize=nltk_analyzer, lower=True, include_lengths=True, batch_first=True, fix_length=300)
LABEL = data.Field(sequential=False, use_vocab=False, is_target=True)

train, test = data.TabularDataset.splits(
      path=imdb_dir, train='train.tsv', test='test.tsv', format='tsv',
      fields=[('Text', TEXT), ('Label', LABEL)])

glove_vectors = Vectors(name=word_embedding_dir + "glove.6B.200d.txt")
TEXT.build_vocab(train, test, vectors=glove_vectors, min_freq=1)

100%|█████████▉| 399932/400000 [00:25<00:00, 15969.63it/s]

In [4]:
import seaborn as sns
length = []
for i in range(len(train)):
  length.append(len(train[i].Text))
df = pd.DataFrame(length, columns=['len'])
df.describe()

,len
count,25000.000000
mean,242.392720
std,180.022367
min,10.000000
25%,132.000000
50%,181.000000
75%,295.000000
max,2525.000000


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE=100
EMBEDDING_DIM = 200
LSTM_DIM = 128
VOCAB_SIZE =TEXT.vocab.vectors.size()[0]
TAG_SIZE = 2
MAX_LEN = 300

# WordEmbedding層
class WordEmbedding(nn.Module):
  def __init__(self, embedding_dim, vocab_size):
    super(WordEmbedding, self).__init__()
    self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.word_embeddings.weight.data.copy_(TEXT.vocab.vectors)
    self.word_embeddings.requires_grad_ = False

  def forward(self, sentence):
    embeds = self.word_embeddings(sentence)
    return embeds

# PositionalEncoder層
class PositionalEncoder(nn.Module):
  def __init__(self, embedding_dim, max_len):
    super(PositionalEncoder, self).__init__()
    self.embedding_dim = embedding_dim
    pe = torch.zeros(max_len, embedding_dim).to(device)

    for pos in range(max_len):
      for i in range(0, embedding_dim, 2):
        pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
        pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim)))
    self.pe = pe.unsqueeze(0)
    self.pe.requires_grad_ = False

  def forward(self, embeds):
    ret = math.sqrt(self.embedding_dim)*embeds + self.pe
    return ret



embedding = WordEmbedding(EMBEDDING_DIM, VOCAB_SIZE).to(device)
positionalEncoder = PositionalEncoder(EMBEDDING_DIM, MAX_LEN).to(device)
loss_function = nn.NLLLoss()

# optimizer = optim.Adam(chain(encoder.parameters(), classifier.parameters()), lr=0.001)

train_iter, test_iter = data.Iterator.splits((train, test), batch_sizes=(BATCH_SIZE, BATCH_SIZE), device=device, repeat=False, sort=False)

In [16]:
losses = []
for epoch in range(1):
    all_loss = 0
    for idx, batch in enumerate(train_iter):
        batch_loss = 0
        embedding.zero_grad()

        text_tensor = batch.Text[0] # (b, s)
        print(text_tensor.size())
        label_tensor = batch.Label

        embeds = embedding(text_tensor) # (b, s, d)
        print(embeds.size())

        pe = positionalEncoder(embeds)
        print(pe.size())

        break
    break

torch.Size([100, 300])
torch.Size([100, 300, 200])
torch.Size([100, 300, 200])
